In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re
import os

In [ ]:
optima = {
    "br17":     39,
    "ft53":   6905,
    "ft70":  38673,
    "ftv33":  1286,
    "ftv35":  1473,
    "ftv38":  1530,
    "ftv44":  1613,
    "ftv47":  1776,
    "ftv55":  1608,
    "ftv64":  1839,
    "ftv70":  1950,
    "ftv90":  1579,
    "ftv100": 1788,
    "ftv110": 1958,
    "ftv120": 2166,
    "ftv130": 2307,
    "ftv140": 2420,
    "ftv150": 2611,
    "ftv160": 2683,
    "ftv170": 2755,
    "kro124p":36230,
    "p43":    5620,
    "rbg323": 1326,
    "rbg358": 1163,
    "rbg403": 2465, 
    "rbg443": 2720,
    "ry48p": 14422,
}

In [ ]:
def annotate_ordering(df: pd.DataFrame) -> pd.DataFrame:
    df["instance_size"] = df.instance.apply(lambda x: int(re.search(r'\d+', x).group()))
    df["method_idx"] = df["method"].apply(lambda x: all_methods.index(x))
    return df.sort_values(by=['instance_size', 'method_idx'])

sa_res_df = pd.read_json('../data/results/sa-out-10.json')
ts_res_df = pd.read_json('../data/results/ts-out-10.json')
all_res_df = pd.concat([sa_res_df, ts_res_df], ignore_index=True)


custom_names = {
    "simulated-annealing": "SA-R",
    "simulated-annealing-nn": "SA-H",
    "tabu-search": "TS-R",
    "tabu-search-nn": "TS-H",
}

all_methods = list(custom_names.values())
all_res_df['method'] = all_res_df['method'].apply(lambda x: custom_names[x])
all_res_df = annotate_ordering(all_res_df).reset_index(drop=True)
selected_instances = all_res_df['instance'].unique()
all_res_df.shape

In [ ]:
SA_METHODS = ["SA-R", "SA-H"]
TS_METHODS = ["TS-R", "TS-H"]

COLORS = {
    "SA-R": "#189990",
    "SA-H": "#094d48",
    "TS-R": "#ff3399",
    "TS-H": "#660033"
}

all_res_df["distance"] = all_res_df.apply(lambda row: (row['cost'] - optima[row['instance']]) / optima[row['instance']], axis=1)
groupped_df = all_res_df.groupby(['instance', 'method'])[["distance", "time", "iterations", "steps", "evaluations", 'cost']].agg(['mean', 'std', 'min', 'max'])
groupped_df.columns = ["_".join(x) for x in groupped_df.columns]
stats_df = groupped_df.reset_index()
stats_df = annotate_ordering(stats_df).reset_index(drop=True)
stats_df.head()

In [ ]:
def plot_violin_by_attr(
        res_df: pd.DataFrame, 
        selected_methods: list[str],
        selected_instances: list[str], 
        attribute: str,
        ax: plt.Axes,
        y_max: int = None,
    ):
    filtered_df = res_df[res_df['method'].isin(selected_methods) & res_df['instance'].isin(selected_instances)]
    sns.violinplot(
        x='instance', 
        y=attribute, 
        hue='method', 
        data=filtered_df, 
        ax=ax,
        linewidth=0, 
        scale='width', 
        palette=COLORS, 
    )
    ax.vlines(
        np.arange(-1, len(selected_instances)) + 0.5, 
        0, 
        filtered_df[attribute].max() if y_max is None else y_max, 
        linestyles='dotted', 
        colors='gray', 
        alpha=0.5,
    )
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    return ax


def plot_bars(
    res_df: pd.DataFrame,
    selected_methods: list[str],
    selected_instances: list[str],
    attribute: str,
    ax: plt.Axes,
    err_attribute: str = None,
    x_attr: str = 'instance',
):
    filtered_df = res_df[res_df['method'].isin(selected_methods) & res_df['instance'].isin(selected_instances)]
    sns.barplot(
        x=x_attr, 
        y=attribute, 
        hue='method',
        data=filtered_df, 
        ax=ax,
        palette=COLORS,
    )
    if err_attribute:
        x_coords = np.array([p.get_x() + p.get_width() / 2 for p in ax.patches])
        y_coords = np.array([p.get_height() for p in ax.patches])
        sorted_idx = np.argsort(x_coords)
        x_coords = x_coords[sorted_idx][:filtered_df.shape[0]]
        y_coords = y_coords[sorted_idx][:filtered_df.shape[0]]
        y_err = filtered_df[err_attribute].to_numpy()
        ax.errorbar(x_coords, y_coords, yerr=y_err, fmt='none', ecolor='black', capsize=2)


    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    return ax

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6), sharex=True)
plot_violin_by_attr(all_res_df, [*TS_METHODS, *SA_METHODS], selected_instances, "distance", ax)
ax.set_ylabel("Distance to optimum")
ax.set_xlabel("Instance")
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
with sns.axes_style("whitegrid"):
    fig, axs = plt.subplots(2, 1, figsize=(12, 6), sharey=True, sharex=True)
    plot_bars(stats_df, [*TS_METHODS, *SA_METHODS], selected_instances, "distance_mean", axs[0], "distance_std")
    axs[0].set_ylabel("Distance")
    axs[0].set_xlabel("")
    axs[0].set_title("Mean")
    axs[0].legend().remove()
    plot_bars(stats_df, [*TS_METHODS, *SA_METHODS], selected_instances, "distance_min", axs[1])
    axs[1].legend().remove()
    axs[1].set_ylabel("Distance")
    axs[1].set_xlabel("Instance")
    axs[1].set_title("Min")
    plt.yscale('log')
    plt.tight_layout()
    plt.show()


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
plot_violin_by_attr(all_res_df, [*SA_METHODS, *TS_METHODS], selected_instances, "time", ax)

ax.legend()
ax.set_ylabel("Running Time")
ax.set_xlabel("Instance")
plt.yscale('log')
plt.yticks([10**i for i in range(0, 13, 3)], ["1ns", "1 us", "1 ms", "1 s", "1000 s"])
plt.tight_layout()
plt.savefig("plots/1/violin_time.png")
plt.show()

In [ ]:
with sns.axes_style("whitegrid"):
    fig, ax = plt.subplots(1, 1, figsize=(12, 4))
    plot_bars(stats_df, TS_METHODS + SA_METHODS, selected_instances, "time_mean", ax, "time_std")
    ax.set_ylabel("Running Time")
    ax.set_xlabel("Instance")
    ax.get_legend().remove()
    
    plt.yscale('log')
    plt.yticks([10**i for i in range(0, 13, 3)], ["1 ns", "1 us", "1 ms", "1 s", "1000 s"])
    plt.tight_layout()
    plt.savefig("plots/1/bar_time.png")
    plt.show()

# SA - per param spec analysis

In [ ]:
rename_dict = {
    "meta-param-1": "temperature",
    "meta-param-2": "cooling_rate",
    "meta-param-3": "markov_chain_length"
}
sa_res_df = all_res_df[all_res_df['method'].isin(SA_METHODS)].rename(columns=rename_dict)
sa_groupped_df = sa_res_df.groupby(['instance', 'method', "temperature", "cooling_rate", "markov_chain_length"])[["distance", "time", "iterations", "steps", "evaluations", 'cost']].agg(['mean', 'std', 'min', 'max'])
sa_groupped_df.columns = ["_".join(x) for x in sa_groupped_df.columns]
sa_stats_df = sa_groupped_df.reset_index()
sa_stats_df = annotate_ordering(sa_stats_df).reset_index(drop=True)

In [ ]:
def plot_stat_by_param(
    df: pd.DataFrame, params: list[str], stat: str, prefix: str, methods: list[str]
):
    for param in params:
        fig, axs = plt.subplots(len(selected_instances) // 4, len(selected_instances) // 2, figsize=(12, 6), sharex=True)
        for instance, ax in zip(selected_instances, axs.flatten()):
            param_df = df.groupby(["instance", "method", param])[[f"{stat}_mean", f'{stat}_std']].mean().reset_index()
            param_df[param] = param_df[param].apply(lambda x: round(x, 2))
            ax.set_title(instance)
            plot_bars(param_df, methods, [instance], f"{stat}_mean", ax, f"{stat}_std", x_attr=param)
            ax.set_ylabel("")
            ax.set_xlabel("")
            ax.get_legend().remove()

        fig.text(-0.01, 0.45, stat.title(), rotation='vertical', fontdict={'fontsize': 12})
        fig.text(0.5, -0.01, param.title().replace("_"," "), ha='center', fontdict={'fontsize': 12})
        plt.tight_layout()
        plt.savefig(f"./plots/2/{prefix}-{param}-{stat}.png")
        plt.show()

In [ ]:
plot_stat_by_param(sa_stats_df, rename_dict.values(), "distance", "sa", SA_METHODS)

In [ ]:
plot_stat_by_param(sa_stats_df, rename_dict.values(), "time", "sa", SA_METHODS)

# TS - per param spec analysis

In [ ]:
rename_dict = {
    "meta-param-1": "patience",
    "meta-param-2": "elite",
    "meta-param-3": "tenure"
}
ts_res_df = all_res_df[all_res_df['method'].isin(TS_METHODS)].rename(columns=rename_dict)
ts_groupped_df = ts_res_df.groupby(['instance', 'method', "patience", "elite", "tenure"])[["distance", "time", "iterations", "steps", "evaluations", 'cost']].agg(['mean', 'std', 'min', 'max'])
ts_groupped_df.columns = ["_".join(x) for x in ts_groupped_df.columns]
ts_stats_df = ts_groupped_df.reset_index()
ts_stats_df = annotate_ordering(ts_stats_df).reset_index(drop=True)

In [ ]:
plot_stat_by_param(ts_stats_df, rename_dict.values(), "distance", "ts", TS_METHODS)

In [ ]:
plot_stat_by_param(ts_stats_df, rename_dict.values(), "time", "ts", TS_METHODS)